In [1]:
%matplotlib inline
from effective_quadratures.PolyParams import PolynomialParam
from effective_quadratures.PolyParentFile import PolyParent
from effective_quadratures.IndexSets import IndexSet
from effective_quadratures.Utils import column, evalfunction
import effective_quadratures.MatrixRoutines as mat
import effective_quadratures.ComputeStats as stats
from effective_quadratures.EffectiveQuadSubsampling import EffectiveSubsampling
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

<h1> Vegetation UQ with Effective-Quadratures

Consider a uniform variation in the following parameters:
    
$$\begin{array}{lll} \hline
Variable & Range & Description \\ \hline
\rho_{shoot} & [38.2, 250.4] & Shoot \; density \; (/m^2)\\ 
L & [15.7, 31.3] & Piston \; blade \; length \; (cm)\\ 
d & [2, 10] & Diameter \;  (mm)\\ 
t & [0.2, 1.0] & Thickness \; (mm)\\  \hline
\end{array}$$

Being by setting up the ranges for the four parameters

In [15]:
quadratic_pts = 3
x1 = PolynomialParam("Uniform", 38.2, 250.4, [], [], [], 3)
x2 = PolynomialParam("Uniform", 15.7, 31.3, [], [], [], 3)
x3 = PolynomialParam("Uniform", 2.0, 10.0, [], [], [], 3)
x4 = PolynomialParam("Uniform", 0.2, 1.0, [], [], [], 3)
orders = [2,2,2,2]
x1x2x3x4 = [x1, x2, x3, x4]

Set the polynomial basis; this will dictate how many function evaluations we will require. Bear in mind that a quadratic in 4 dimensions requires $3^4=81$ points. To reduce the cost we will opt for a total order basis.

In [18]:
polybasis = IndexSet("total order", orders)
print IndexSet.getIndexSet(polybasis)
maximum_number_of_evals = IndexSet.getCardinality(polybasis)
print maximum_number_of_evals

[[ 0.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  0.  2.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  1.]
 [ 0.  0.  2.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  1.  0.  1.]
 [ 0.  1.  1.  0.]
 [ 0.  2.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 1.  0.  0.  1.]
 [ 1.  0.  1.  0.]
 [ 1.  1.  0.  0.]
 [ 2.  0.  0.  0.]]
15


In [19]:
effectiveQuads = EffectiveSubsampling(x1x2x3x4, polybasis, 0)
A, esquad_pts, W, indices = EffectiveSubsampling.getAsubsampled(effectiveQuads, maximum_number_of_evals)
print esquad_pts

[[ 144.3          23.5           6.            0.6       ]
 [  62.11529339   17.45814598    2.90161332    0.6       ]
 [ 226.48470661   29.54185402    6.            0.29016133]
 [  62.11529339   23.5           9.09838668    0.90983867]
 [ 226.48470661   23.5           2.90161332    0.90983867]
 [ 226.48470661   17.45814598    6.            0.29016133]
 [  62.11529339   29.54185402    6.            0.29016133]
 [ 226.48470661   23.5           9.09838668    0.90983867]
 [ 144.3          29.54185402    6.            0.90983867]
 [ 144.3          23.5           9.09838668    0.29016133]
 [ 144.3          17.45814598    6.            0.90983867]
 [ 144.3          23.5           2.90161332    0.29016133]
 [  62.11529339   17.45814598    6.            0.29016133]
 [ 144.3          17.45814598    9.09838668    0.6       ]
 [  62.11529339   23.5           2.90161332    0.90983867]]
